In [4]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import cv2 as cv

In [5]:
def remap(img,x,y):
    """ a wrapper of opencv remap, adopted from 
    img:NxHxW, x:NxHxW ,y:NxHxW
    out:NxHxW
    Implement Scaling and Squaring
    https://github.com/yongleex/DiffeomorphicPIV/blob/main/deformpiv.py#L180
    """
    
    # convert x,y to grid:NxHxWx2
    grid = torch.stack((x, y), dim=-1)
    
    # normalize grid to (-1,1) for grid_sample
    # under pixel coordination system, x->W, y->H
    grid_shape = grid.shape[1:3]
    grid[:,:,:,0] = (grid[:,:,:,0] / (grid_shape[1] - 1) - 0.5)*2
    grid[:,:,:,1] = (grid[:,:,:,1] / (grid_shape[0] - 1) - 0.5)*2

    # shape img to NxCxHxW for grid_sample
    img = torch.unsqueeze(img, dim=1)
    out = F.grid_sample(img, grid, mode='bicubic', align_corners=True)
    
    return torch.squeeze(out, dim=1)

In [6]:
path = 'data/0016.npy'

data = np.load(path)
print(data.shape)
data_tensor = torch.tensor(data)
image1 = data[0]
image2 = data[1]
u = data[2]
v = data[3]
# image1 = (image1 * 255.0).astype('uint8')
# image2 = (image2 * 255.0).astype('uint8')
# u = (u * 255.0).astype('uint8')
# v = (v * 255.0).astype('uint8')

# 使用Lee的方法
image1_tensor, image2_tensor, u_tensor, v_tensor = data_tensor.split(1, 0)
re = remap(image1_tensor, u_tensor, v_tensor)
re = (re + image1_tensor)[0].numpy()
# re_image = re.astype('uint8')

# print(re_image.shape)
# plt.imsave('111.png', re_image)
print(re.shape)
plt.imsave('111.png', re)


# 使用opencv的方法
# re = cv.remap(image1, u, v, interpolation = 4)
# re = image1 + re
# print(re.shape)
# plt.imsave('111.png', re)

# 使用Lee的方法,改变数值
# image1_tensor, image2_tensor, u_tensor, v_tensor = data_tensor.split(1, 0)
# # image1_tensor = image1_tensor * 20.0
# # u_tensor = u_tensor * 10.0
# # v_tensor = v_tensor * 10.0
# re = remap(image1_tensor, u_tensor, v_tensor)
# re = re[0].numpy()
# re_image = (re * 255.0).astype('uint8')
# print(re_image.shape)
# plt.imsave('111.png', re_image)

(4, 440, 1024)
(440, 1024)
